# Задание 1 (поиск подстроки в строке)

In [7]:
s = input("Строка ")
subs = input("Подстрока ")
case = input("Введите 1 чтобы опустить регистр ")
if (case == '1'):
    subs = subs.lower() 
    subs = subs.strip()
    s = s.lower()
    s = s.strip()
else:
    s = s.strip()
    subs = subs.strip()

Строка abbbc
Подстрока bc
Введите 1 чтобы опустить регистр 


In [8]:
def kmp_table(W):
    n = len(W)
    T = [0 for i in range(n)]
    T[0] = -1 
    pos = 0

    for i in range(1, n):
        if W[i] == W[pos]:#
            T[i] = T[pos]
        else:
            T[i] = pos
            pos = T[pos]
            while pos >= 0 and W[i] != W[pos]:
                pos = T[pos]
        pos += 1

    return T
def KMP(string, substring):
    m = len(substring)
    n = len(string)
    T = kmp_table(substring)

    pos = 0
    i = 0
    while i < n:
        if string[i] != substring[pos]:
            pos = T[pos]
            if pos < 0:
                pos += 1
                i += 1
        else:
            i += 1
            pos += 1

            if pos == m:
                return True
    return False
if (KMP(s,subs)):
    print("Вхождение есть")
else:
    print("Не найдено")

Вхождение есть
Вхождение есть


In [9]:
NO_OF_CHARS = 256
 
def badCharHeuristic(string, size):
    badChar = [-1]*NO_OF_CHARS
    for i in range(size):
        badChar[ord(string[i])] = i;
    return badChar
 
def search(txt, pat):
    m = len(pat)
    n = len(txt)
    badChar = badCharHeuristic(pat, m)
    s = 0
    while(s <= n-m):
        j = m-1
        while j>=0 and pat[j] == txt[s+j]:
            j -= 1
        if j<0:
            print("Совпадение на позиции = {}".format(s))
            s += (m-badChar[ord(txt[s+m])] if s+m<n else 1)
        else:
            s += max(1, j-badChar[ord(txt[s+j])])
def main():
    search(s, subs)
if __name__ == '__main__':
    main()

Совпадение на позиции = 3
Совпадение на позиции = 3


# Задание "пятнашки"

In [10]:
from math import sqrt
from heapq import heappop, heappush

def a_star(start_chain, goal_node):
    node_hash = {} # словарь для записи передвижений каждой секции
    chains_queue = [] # очередь секций
    heappush(chains_queue, start_chain) # добавление очереди
    while chains_queue: # цикл очереди
        cur_chain = heappop(chains_queue) # берем секцию из очереди
        cur_node = cur_chain.last_node() # получение события на поле в строке
        if cur_node == goal_node: # условие завершения 15-ек, они упорядочны
            return cur_chain
        node_hash[cur_node] = cur_chain.g() # записываем длину шагов для этой секции
        for chain in cur_chain.get_neighbours():
            if chain.last_node() in node_hash:
                if chain.g() >= node_hash[chain.last_node()]:
                    continue
                node_hash[chain.last_node()] = chain.g()
            heappush(chains_queue, chain)

# Нахождение расстояния между а и b в матрице размера n
def dst_matrix(a, b, n):
    return abs(a % n - b % n) + abs(a // n - b // n)

# Класс пятнашек
class chain15:
    # Функция вывода поля пятнашек
    def __str__(self):
        i = 0
        matrix = ""
        while i < self.size ** 2:
            matrix += str(self.board_state[i]) + " "
            if i % self.size == 3:
                matrix += "\n"
            i += 1
        return matrix

    # Инициализация

    def __init__(self, board_state, history=[]):
        self.board_state = list(board_state)
        self.size = int(sqrt(len(board_state)))
        self.history = history
        self.quad_size = int(self.size * self.size)

    def manh_dst(self):
        dst = 0
        for i in range(0, self.quad_size):
            dst += dst_matrix((self.board_state[i] - 1) % self.quad_size, i, self.size)
        return int(dst)

    def last_node(self):
        # Должен являться хеширующемся значением. Список не хешируем.
        return str(self.board_state)

    # Один линейный конфликт увеличивает эвристику на 2. Здесь учитываются передвижения вверх и вниз.
    # Передвижения влево и вправо заложены в манхеттенское расстоянии
    def linear_conflict(self):
        conflict_count = 0
        return 2 * conflict_count

    def last_move(self):
        if self.board_state[-1] == self.quad_size - 1 or self.board_state[-1] == self.quad_size - self.size:
            return 0
        return 2

    # Рассматриваем угловые ячейки
    def corner_tiles(self):
        conflict_count = 0
        # Левый верхний угол
        if self.board_state[0] != 1:
            if self.board_state[1] == 2 or self.board_state[self.size] == self.size + 1:
                conflict_count += 1
        # Правый верхний угол
        if self.board_state[self.size - 1] != self.size:
            if self.board_state[self.size - 2] == self.size - 1 or self.board_state[self.size * 2 - 1] == self.size * 2:
                conflict_count += 1
        # Левый нижний угол
        if self.board_state[self.quad_size - self.size] != self.quad_size - self.size + 1:
            if self.board_state[self.quad_size - self.size * 2] == self.quad_size - self.size * 2 + 1 or self.board_state[self.quad_size - self.size + 1] == self.quad_size - self.size + 2:
                conflict_count += 1
        # Правый нижний угол не рассматривается, т.к. он остается пустым
        return 2 * conflict_count

    def simple_heur(self):
        dst = 0
        for i in range(0, int(self.quad_size)):
            if (self.board_state[i] - 1) != i:
                dst += 1
        return dst

    def h(self):
        return self.manh_dst() + self.last_move()

    def g(self):
        return len(self.history)

    def f(self):
        return self.h() + self.g()

    def __lt__(self, other):
        return self.f() < other.f()

    def get_neighbours(self):# получение соседних элементов
        neighs = []
        zero_coord = self.board_state.index(0)
        if zero_coord + 1 < self.size ** 2 and dst_matrix(zero_coord, zero_coord + 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + 1] = new_state[zero_coord + 1], new_state[zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))


        if zero_coord - 1 >= 0 and dst_matrix(zero_coord, zero_coord - 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - 1] = new_state[zero_coord - 1], new_state[zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))


        if zero_coord + self.size < self.size ** 2 and dst_matrix(zero_coord, zero_coord + self.size, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + self.size] = new_state[zero_coord + self.size], new_state[
                zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))


        if zero_coord - self.size >= 0 and dst_matrix(zero_coord, zero_coord - self.size, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - self.size] = new_state[zero_coord - self.size], new_state[
                zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        return neighs


if __name__ == '__main__':# функция проверки
    turns_to_make = []
    start_state = list(map(int, input("Начальный массив: ").split(" ")))
    summ = 0
    for i in range(16):
        current_number = start_state[i]
        for j in range(i+1, 16):
            if start_state[j] < current_number and start_state[j] != 0:
                summ += 1
    e = start_state.index(0) // 4 + 1
    summ += e
    if summ % 2 == 1:
        print(turns_to_make)
    else:
        start = chain15(start_state)
        end = chain15((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0))
        result = a_star(start, end.last_node())
        node_first = result.history[0]
        turns_to_make = []
        for node in result.history:
            for j in range(len(node.board_state)):
                if node_first.board_state[j] != node.board_state[j] and node.board_state[j] != 0:
                    turns_to_make.append(node.board_state[j])
            node_first = node
        if turns_to_make[len(turns_to_make) - 1] == 14:
            turns_to_make.append(15)
        else:
            turns_to_make.append(12)
        print(turns_to_make)

Начальный массив: 1 2 3 4 5 6 7 8 13 9 11 12 10 14 15 0
[15, 14, 10, 13, 9, 10, 14, 15]
[15, 14, 10, 13, 9, 10, 14, 15]
